In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.dummy import DummyClassifier
from joblib import Parallel, delayed
from joblib_progress import joblib_progress
from datetime import datetime
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
import numpy as np
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline as IMBPipeline
from sklearn.feature_selection import RFE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import extract_features
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore') 


In [2]:
model_name = 'neuralmind/bert-base-portuguese-cased'

random_seed = 42

raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'
results_cr_path = '../reports/classification_reports/'
test_results_path = '../reports/test_results/'
reports_path = '../reports/'

target_list = ['ig','bo', 'cl', 'co', 'gl', 'lu']

In [3]:
top_ment_time_path = raw_data_path + '{}_r3_{}_top_mentioned_timelines.csv'
list_train_paths_tmt = [top_ment_time_path.format("train",t) for t in target_list]
list_test_paths_tmt = [top_ment_time_path.format("test",t) for t in target_list]

users_path = raw_data_path + 'r3_{}_{}_users.csv'
list_train_paths_users = [users_path.format(t,"train") for t in target_list]
list_test_paths_users = [users_path.format(t,"test") for t in target_list]

In [4]:

train = pd.read_csv(list_train_paths_users[0], **{'sep': ';', 'encoding': 'utf-8-sig'})
test = pd.read_csv(list_test_paths_users[0], **{'sep': ';', 'encoding': 'utf-8-sig'})

In [29]:
X_train = train.Stance.values
y_train = train.Polarity.values
X_test = test.Stance.values
y_test = test.Polarity.values

In [16]:
model_name = 'pablocosta/bertabaporu-base-uncased' 

In [35]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [36]:
from transformers import AutoTokenizer, BertForSequenceClassification  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
#model = AutoModelForPreTraining.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
X_train

array(['tenho pra mim que grande parte senão todas as patologias sociais atuais são culpa da igreja (misoginia, homofobia, racismo etc) ainda veio o protestantismo misturar cristianismo com capitalismo as duas escórias da humanidade sou obrigada a ter empatia',
       'Cidade de Deus Alicate: quer saber vou entrar pra igreja, essa vida de bicho solto é pra maluco',
       'Acordei já sendo removida do grupo da igreja', ...,
       'O BRASIL APRENDEU A VOTAR COMPRA O APÊ, THELMA, SAI DO ALUGUEL FODA-SE CRENTE CONSTRUINDO IGREJA NA ÁFRICA',
       'evangelização de adolescentes em uma igreja evangélica foi horrível 8 da manhã do domingo eu sentada numa cadeira parecendo que tava em aula e todo mundo lendo a bíblia e amando deus',
       'Imagina se o dono dessa igreja fisga o Neymar. Um dízimo de 10% sobre quase 15 milhões todo mês . Baita negócio'],
      dtype=object)

In [44]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}
    
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits
    
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return "positive" if preds.item() == 1 else "negative"

In [45]:
# Set up parameters
bert_model_name = 'pablocosta/bertabaporu-base-uncased' 
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [31]:
for text in X_train:
    
    tokens = tokenizer(text, add_special_tokens=False, truncation=False, return_tensors="pt")